In [21]:
import json
import glob, os
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [22]:
data_dir="C:\\Users\\tsb7592\\Downloads\\Clean Excels"

In [23]:
os.chdir(data_dir)

In [24]:
all_files = glob.glob('C:\\Users\\tsb7592\\Downloads\\Clean Excels\\*')

In [25]:
doc =[]
for i in range(len(all_files)):
    data = pd.read_excel(all_files[i])
    data['Pin'] = i
    doc.append(data)
df = pd.concat(doc)

In [26]:
df.shape

(326233, 99)

In [27]:
df['Pin'].nunique()

93

In [8]:
mni = pd.read_excel(all_files[23])

In [9]:
list(df.soundFilename.notnull().unique())

[False, True]

In [10]:
mni['testHandcoding'].unique()

array([nan, 'B', 'too dark to code', 'away', 'right', 'left'],
      dtype=object)

In [11]:
mni['calibHandcoding'].value_counts()

TL    40
BL    33
TR    33
BR    30
Name: calibHandcoding, dtype: int64

In [12]:
mni=mni.rename(columns = {'timestamp':'elapsedTime'})

# Read json file

In [35]:
import json
import os
import pandas as pd

In [36]:
cols=['eventName','gazeLocationName', 'calibHandcoding', 'testHandcoding', 'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',"lookAtPointX", "lookAtPointY",]

In [37]:
df=df.rename(columns = {'timestamp':'elapsedTime'})

In [16]:
df.columns

Index(['elapsedTime', 'testState', 'eventName', 'dataKey', 'dataValue',
       'gazeLocationName', 'calibHandcoding', 'testHandcoding',
       'gazeLocationPercentX', 'gazeLocationPercentY',
       'calibrationPoint1AverageX', 'calibrationPoint1AverageY',
       'calibrationPoint2AverageX', 'calibrationPoint2AverageY',
       'calibrationPoint3AverageX', 'calibrationPoint3AverageY',
       'calibrationPoint4AverageX', 'calibrationPoint4AverageY',
       'cameraImageFilename', 'itemNumber', 'leftImageFilename',
       'rightImageFilename', 'soundFilename', 'spokenWord',
       'calibrationFocalPoint', 'lookAtPointX', 'lookAtPointY', 'facePointX',
       'facePointY', 'arRightEyeTransformRow3Column3',
       'arHitTestLookAtPointPositionY', 'arLeftEyeTransformRow1Column3',
       'arLeftEyeTransformRow3Column2', 'arFacePositionX',
       'arFaceAnchorTransformRow0Column0', 'arLeftEyeTransformRow1Column0',
       'arRightEyeTransformRow0Column0', 'arRightEyeTransformRow2Column1',
       '

# Rewrite Lookalgorithm

In [17]:
import math
import numpy as np

def lookAlgorithm(df, usevals='lookAtPoint', searchTime=1.0167, margX=10, margY=15, alignX=30, alignY=30):
    '''
    Input: 
        df: the json file read to a pandas dataframe
        usevals: a string of the column name that is used to identify the position
    Output:
         results in a pandas dataframe
    '''
    
    if 'lookAtPoint' in usevals:
        useX = "lookAtPointX"
        useY = "lookAtPointY"
    elif 'arHitTestLookAtPointPosition' in usevals:
        useX = "arHitTestLookAtPointPositionX"
        useY = "arHitTestLookAtPointPositionY"
    
    startRows =list(df[df['eventName'].str.contains('CalibrationBegan')].index)
    endRows =list(df[df['eventName'].str.contains('CalibrationEnded')].index)
    
    if len(startRows) != len(endRows):
        print("Early-terminated Calibration")
        return
    
    RMs=[]
    for i in range(len(startRows)):
        rollingMeans = pd.DataFrame(columns=('Cali','calibFocal', 'rowIndex','rollingX','rollingY','props')) 
        sub = df.iloc[startRows[i]:endRows[i]+1]
        firstCheck_time = df.iloc[startRows[i]]['elapsedTime'] + searchTime
        firstCheck_row = sub[sub['elapsedTime'] <= firstCheck_time].index.max()
       
        for j in range(firstCheck_row, endRows[i]+1):
            rollingMeans.loc[j] = [i,'tmp', j, -9999,-9999, -9999]
            
            useData =sub[(sub['elapsedTime'] >= df.iloc[j]['elapsedTime'] - searchTime) & (sub['elapsedTime'] <= df.iloc[j]['elapsedTime'])&(sub['elapsedTime'] >= df.iloc[startRows[i]]['elapsedTime'])]
            useData = useData[~useData.eventName.str.contains('ended')]
                              
            rollingMeans.loc[j]['calibFocal']=useData['calibrationFocalPoint'].unique()[0]
            rollingMeans.loc[j]['rollingX']=round(useData[useX].mean(),2)
            rollingMeans.loc[j]['rollingY']=round(useData[useY].mean(),2)
            if useData[useData[useY].notnull()].shape[0] == 0:
                rollingMeans.loc[j]['props'] =-9999
            else:
                rollingMeans.loc[j]['props']=round(100*(useData[(useData[useX] >= useData[useX].mean()- margX) & (useData[useX] <= useData[useX].mean()+ margX) & (useData[useY] >= useData[useY].mean()- margY) & (useData[useY] <= useData[useY].mean()+ margY)]).shape[0]/useData[useData[useY].notnull()].shape[0],2)

        RMs.append(rollingMeans)
         
    RMs = pd.concat(RMs, ignore_index=True)
    
    use=[]
    for i in range(4):
        ur = RMs[(RMs['props'] >= 80) & (RMs['Cali']==i)].head(1)
        use.append(ur)
    useRows = pd.concat(use)
    useRows = useRows.reset_index()
    
    li = ['topRight','bottomRight','topLeft','bottomLeft']
    if useRows.shape[0]==3:
        missing = np.setdiff1d(li, list(useRows['calibFocal'].unique()))[0]
        if missing == 'topLeft':
            useRows.loc[3]=[np.NaN,4, missing, np.NaN, useRows[useRows['calibFocal']=='bottomLeft']['rollingX'].values[0],useRows[useRows['calibFocal']=='topRight']['rollingY'].values[0], 81]
        elif missing =='topRight':
            useRows.loc[3]=[np.NaN, 4, missing, np.NaN, useRows[useRows['calibFocal']=='bottomRight']['rollingX'].values[0],useRows[useRows['calibFocal']=='topLeft']['rollingY'].values[0], 81]
        elif missing =='bottomLeft':
            useRows.loc[3]=[np.NaN, 4, missing, np.NaN, useRows[useRows['calibFocal']=='topLeft']['rollingX'].values[0],useRows[useRows['calibFocal']=='bottomRight']['rollingY'].values[0], 81]
        elif missing =='bottomRight':
            useRows.loc[3]=[np.NaN, 4, missing, np.NaN, useRows[useRows['calibFocal']=='topRight']['rollingX'].values[0],useRows[useRows['calibFocal']=='bottomLeft']['rollingY'].values[0], 81]

    if useRows.shape[0]<3:
        return 2
    
    useRows['calibrationSet'] =(useRows.index + 1)/4
    useRows['calibrationSet'] =useRows['calibrationSet'].apply(np.ceil)
    
    tr = useRows[useRows['calibFocal']=='topRight']['rollingX'].values[0]
    br = useRows[useRows['calibFocal']=='bottomRight']['rollingX'].values[0]
    tl = useRows[useRows['calibFocal']=='topLeft']['rollingX'].values[0]
    bl = useRows[useRows['calibFocal']=='bottomLeft']['rollingX'].values[0]    
    
    #save the 4 screenSide numbers to a dictionary 
    screenSide={}

    if abs(tr- br) <= alignX:
        screenSide['rightSide'] = max(tr, br)
    else:
        screenSide['rightSide'] = np.NaN
    
    if abs(tl- bl) <= alignX:
        screenSide['leftSide']= min(tl, bl)
    else:
        screenSide['leftSide']= np.NaN
    
    tr = useRows[useRows['calibFocal']=='topRight']['rollingY'].values[0]
    br = useRows[useRows['calibFocal']=='bottomRight']['rollingY'].values[0]
    tl = useRows[useRows['calibFocal']=='topLeft']['rollingY'].values[0]
    bl = useRows[useRows['calibFocal']=='bottomLeft']['rollingY'].values[0]
    
    if 'lookAtPoint' in usevals:
        if abs(tr- tl) <= alignY:
            screenSide['bottomSide'] = round((tr+tl)/2,4)
        else:
            screenSide['bottomSide'] = np.NaN
        
        if abs(br- bl) <= alignY:
            screenSide['topSide']= round((br+bl)/2,4)
        else:
            screenSide['topSide']= np.NaN   
        
    elif 'arHitTestLookAtPointPosition' in usevals:
        if abs(tr- tl) <= alignY:
            screenSide['topSide'] = round((tr+tl)/2,4)
        else:
            screenSide['topSide'] = np.NaN
        
        if abs(br- bl) <= alignY:
            screenSide['bottomSide']= round((br+bl)/2,4)
        else:
            screenSide['bottomSide']= np.NaN   
        
    if useRows['calibFocal'].isnull().any():
        screenSide['rightSide'] = np.NaN
        screenSide['leftSide'] = np.NaN
        screenSide['topSide'] = np.NaN
        screenSide['bottomSide'] = np.NaN
 
    return screenSide

In [18]:
count2 = 0
DC ={}
for i in range(93):
    data = df[df.Pin==i]
    x = lookAlgorithm(data)
    if x ==2:
        count2+=1
    elif np.nan in x.values():
        continue
    else:
        DC[i] =x

In [19]:
DC

{6: {'rightSide': 120.61,
  'leftSide': 14.7,
  'bottomSide': -12.605,
  'topSide': -6.75},
 10: {'rightSide': 75.0,
  'leftSide': 22.55,
  'bottomSide': 7.86,
  'topSide': 19.54},
 13: {'rightSide': -164.42,
  'leftSide': -199.64,
  'bottomSide': 174.895,
  'topSide': 177.36},
 15: {'rightSide': 183.53,
  'leftSide': 84.46,
  'bottomSide': 0.28,
  'topSide': 15.635},
 17: {'rightSide': 110.78,
  'leftSide': 42.04,
  'bottomSide': 18.65,
  'topSide': -14.01},
 20: {'rightSide': 112.19,
  'leftSide': 21.24,
  'bottomSide': 49.52,
  'topSide': 69.62},
 22: {'rightSide': 140.8,
  'leftSide': 63.66,
  'bottomSide': 13.985,
  'topSide': 19.97},
 23: {'rightSide': 187.66,
  'leftSide': 83.31,
  'bottomSide': 46.995,
  'topSide': 56.855},
 24: {'rightSide': 167.01,
  'leftSide': 88.44,
  'bottomSide': 33.4,
  'topSide': 35.59},
 28: {'rightSide': 128.82,
  'leftSide': 89.58,
  'bottomSide': -14.37,
  'topSide': -6.81},
 30: {'rightSide': 173.5,
  'leftSide': 90.26,
  'bottomSide': -16.645,
  

In [20]:
len(DC)

33

In [28]:
Select = list(DC.keys())
Select

[6,
 10,
 13,
 15,
 17,
 20,
 22,
 23,
 24,
 28,
 30,
 33,
 34,
 36,
 40,
 45,
 47,
 48,
 49,
 51,
 53,
 56,
 62,
 72,
 73,
 76,
 80,
 83,
 86,
 89,
 90,
 91,
 92]

## Modify the numbers

In [39]:
def Modify_screenSide(ss, ratio= 0.4, leftright= 25, topbottom=50):
    '''
    Input: 
        ss: dictionary contains the four screensides
        leftright: the margin to modify the left/right side
        topbottom: the margin to modify the top/bottom side
        ratio: the ratio to identify left and right side
        
    Output:
        screenSide: the modified dictionary contains new four screensides
    '''
    screenSide =ss.copy()
    screenSide['rightSide'] += leftright
    screenSide['leftSide'] -= leftright
    screenSide['topSide'] += topbottom
    screenSide['bottomSide'] -= topbottom
    
    return screenSide

In [40]:
def Location(row, nss, ratio, useX,useY):
    '''
    Input: 
        row: a row of a pandas dataframe
        nss: the modified dictionary contains new four screensides
        ratio: the ratio to identify left and right side
        useX: column name contains the x
        useY: column name contains the y 

    '''        
    
    leftM =  nss['leftSide']+(nss['rightSide']-nss['leftSide'])* ratio
    rightM = nss['rightSide']-(nss['rightSide']-nss['leftSide'])*ratio
    if row[useX]  >= nss['leftSide'] and row[useX] <=leftM  and row[useY]<=nss['topSide']and row[useY]>=nss['bottomSide'] :
        return 'left'
    elif row[useX] >= rightM  and row[useX] <=nss['rightSide'] and row[useY]<=nss['topSide']and row[useY]>=nss['bottomSide']:
        return 'right'
    else:
        return 'away'

In [41]:
def OnScreen(row, nss, ratio, useX, useY):
    '''
    Input: 
        row: a row of a pandas dataframe
        nss: the modified dictionary contains new four screensides
        ratio: the ratio to identify left and right side
        useX: column name contains the x
        useY: column name contains the y 
    '''                
    leftM =  nss['leftSide']+(nss['rightSide']-nss['leftSide'])* ratio
    rightM = nss['rightSide']-(nss['rightSide']-nss['leftSide'])*ratio
    if row[useX] >=leftM and row[useX]<= rightM and row[useY]>=nss['bottomSide']and row[useY]<=nss['topSide'] :
        return 'true'
    else:
        return 'false'

In [103]:
def Label(row, var='x'):
    '''
    Input: 
        row: a row of a pandas dataframe
        var: column name contains x/y 
    '''      
    if var =='x':
        total = row[['left_x', 'away_x', 'right_x']].sum()
        left = 'left_x'
        right = 'right_x'
        away = 'away_x'
    else:
        total = row[['left_y', 'away_y', 'right_y']].sum()
        left = 'left_y'
        right = 'right_y'
        away = 'away_y'   
        
    if row[away] > total*0.7:
        return 'unscoreable'
    
    elif row[left] > total*0.5:
        return 'left'
    
    elif row[right] > total*0.5:
        return 'right'    
    
    elif row[left] - row[right] > total*0.1:
        return 'left'
    
    elif row[right] -row[left] > total*0.1:
        return 'right'    
    
    else:
        return 'unscoreable'


In [43]:
cols=['eventName', 'gazeLocationName',
      'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',
 'lookAtPointX', 'lookAtPointY','NewLocation','NewOnScreen']

In [69]:
#TSV file
def count_original(df):
    '''
    Input: 
        df: the json file read to a pandas dataframe

    Output:
         obs: the pandas dataframe contains the original count from the json
    '''
    df = df[~df['itemNumber'].isnull()]
    df['calibHandcoding'] = df['calibHandcoding'].str.strip()
    df['testHandcoding'] = df['testHandcoding'].str.strip()
    obs = df.groupby('itemNumber')['calibHandcoding'].value_counts().unstack().reset_index()
    obs1 = df.groupby('itemNumber')['testHandcoding'].value_counts().unstack().reset_index()
    obs1 =obs1[obs1.itemNumber >=5 ]
    obs = obs.iloc[:4,:]
    Lcol = obs.filter(like='L').columns
    Rcol = obs.filter(like='R').columns
    
    obs['left']=obs[Lcol].sum(axis=1)
    obs['right']=obs[Rcol].sum(axis=1)
    if 'way' in obs.columns:
        obs['away']=obs['away']+obs['way']
    if 'away' not in obs.columns:
        obs['away'] = 0
        
    obs= obs.fillna(0)
    obs1= obs1.fillna(0)
    
    keep =['left', 'away',  'right']
    rv= obs[keep].append(obs1[keep])
    rv.reset_index()

    return rv[keep]

In [45]:
#json file

def ReProcess(data, useX, useY,  nss, ratio):
    '''
    Input: 
        data: the json file read to a pandas dataframe
        useX: column name contains the x
        useY: column name contains the y 
        nss: the modified dictionary contains new four screensides
        ratio: the ratio to identify left and right side
        
    Output:
         new: the pandas dataframe contains the new count 
         items: a list of the name of the N items
    '''

    #Finding the index for presentLiveItem and completeLiveItem to subset the pieces we need for analysis
        
    df = data.copy()
    df= df[(df[useX].notnull()) & (df[useY].notnull())]
    df['NewLocation'] = df.apply( lambda row : Location(row, nss, ratio, useX, useY), axis = 1)
    
    df = df[~df['itemNumber'].isnull()]
    items=list(df.soundFilename.unique())
    new = df.groupby('itemNumber')['NewLocation'].value_counts().unstack().reset_index()
    keep =['left', 'away', 'right', 'itemNumber']
    
    return new[keep], df, items

# Wrapper

In [46]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline


def plot_dots(df, title, ss, nss, ratio, useX, useY):
    '''
    Input: 
        df: the pandas dataframe contains the NewLocation, NewOnScreen
        title: the string of the item name
        nss: the modified dictionary contains new four screensides
        ratio: the ratio to identify left and right side
        useX: column name contains the x
        useY: column name contains the y       
        
    Output:
        a plot for the item 
    '''
    leftM =  nss['leftSide']+(nss['rightSide']-nss['leftSide'])* ratio
    rightM = nss['rightSide']-(nss['rightSide']-nss['leftSide'])*ratio

    width = ss['rightSide'] - ss['leftSide']
    height = abs(ss['topSide'] - ss['bottomSide'])
    
    w1= (nss['rightSide']-nss['leftSide'])* ratio
    h1= abs( nss['topSide']-nss['bottomSide'])
    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, aspect='equal')
    
    #color by different category
    OA =df[df['NewLocation'] == 'away']
    ax1.scatter(x=OA[useX], y=OA[useY],alpha=0.9, color='black',marker='+')
    
    LR =df[(df['NewLocation'] == 'left') | (df['NewLocation']=='right')]
    ax1.scatter(x=LR[useX], y=LR[useY],alpha=0.9, color='orange', marker='.')
    
    ax1.add_patch(patches.Rectangle((nss['leftSide'], min(nss['bottomSide'], nss['topSide'])), w1, h1,color='red',fill=False))
    ax1.add_patch(patches.Rectangle((rightM, min(nss['bottomSide'],nss['topSide'])), w1, h1,color='red',fill=False))
    
    ax1.add_patch( patches.Rectangle((ss['leftSide'], ss['bottomSide']), width, height, color='green', fill=False))
    plt.title(title)
    
    plt.gca().invert_yaxis()
    plt.show()

https://researchpy.readthedocs.io/en/latest/crosstab_documentation.html

In [109]:
import researchpy

def Run_different(df, key, usevals='lookAtPoint'):
    '''
    Input: 
        df: the json pandas dataframe
        key: the key identifying which participant
        usevals: a string of the column name that is used to identify the position
        
    Output:
        saved the dataframe for each given margin
    '''
    rv = []
    if 'lookAtPoint' in usevals:
        a = 25
        b = 50
        margX = 10
        margY = 15
        alignX = 30
        alignY = 30
        useX = "lookAtPointX"
        useY = "lookAtPointY"
        
    elif 'arHitTestLookAtPointPosition' in usevals:
        a = 0.025
        b = 0.05
        margX = 0.01
        margY = 0.015
        alignX = 0.03
        alignY = 0.03
        useX = "arHitTestLookAtPointPositionX"
        useY = "arHitTestLookAtPointPositionY"   

    else:
        return 'Choose lookAtPoint or arHitTestLookAtPointPosition'
     
    #Using the following three margins to test    
    margins = [0.4 , 0.425, 0.45] 
    ss = lookAlgorithm(df, usevals, 1.0167, margX, margY, alignX, alignY)
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    #filedir ='C:/Users/tsb7592/Box/Baby Toolbox Operations/Analysis/Hdata/'+str(key)+'_counts.xlsx'
    #writer = pd.ExcelWriter(filedir, engine='xlsxwriter')

    obs = count_original(df)
    obs['itemNumber'] = range(1, 15)
    
    ct = []
    rv =[]
    for mar in margins:
        nss= Modify_screenSide(ss, mar, a,b)
        new, dfm , items = ReProcess(df, useX, useY, nss, mar)

        # Write each dataframe to a different worksheet
        comb = pd.merge(obs,new, how="left", on='itemNumber')
        comb = comb.fillna(0)
        comb['Label_x'] = comb.apply( lambda row : Label(row, 'x'), axis = 1)
        comb['Label_y'] = comb.apply( lambda row : Label(row, 'y'), axis = 1)
        ordr =['left_x', 'away_x', 'right_x', 'Label_x','left_y', 'away_y', 'right_y','Label_y']
        #comb[ordr].to_excel(writer, sheet_name=str(mar), index=False)
        
        #Skip the first 4 calibration rows
        data = comb[ordr].loc[4:]
        crosstab, res =researchpy.crosstab(data['Label_x'], data['Label_y'], test= "chi-square")
        rv.append(res)
        ct.append(crosstab)
   # writer.save()
    rv = pd.concat(rv,axis=1)
    ct = pd.concat(ct)
    display(rv)
    return ct

In [106]:
Select[0]

6

In [110]:
sd ={}
for sid in Select:
    subset = df[df.Pin==sid]
    print('Number ',sid)
    ct =Run_different(subset, sid,'lookAtPoint' )  
    sd[sid]=ct
    print('-----------------------------------------------------------------------')

Number  6


,Chi-square test,results,Chi-square test,results,Chi-square test,results
0,Pearson Chi-square ( 2.0) =,10.0000,Pearson Chi-square ( 2.0) =,10.0000,Pearson Chi-square ( 2.0) =,10.0000
1,p-value =,0.0067,p-value =,0.0067,p-value =,0.0067
2,Cramer's V =,1.0000,Cramer's V =,1.0000,Cramer's V =,1.0000


-----------------------------------------------------------------------
Number  10


,Chi-square test,results,Chi-square test,results,Chi-square test,results
0,Pearson Chi-square ( 4.0) =,2.6667,Pearson Chi-square ( 4.0) =,2.6667,Pearson Chi-square ( 4.0) =,2.6667
1,p-value =,0.6151,p-value =,0.6151,p-value =,0.6151
2,Cramer's V =,0.3651,Cramer's V =,0.3651,Cramer's V =,0.3651


-----------------------------------------------------------------------
Number  13


...

-----------------------------------------------------------------------
Number  91


,Chi-square test,results,Chi-square test,results,Chi-square test,results
0,Pearson Chi-square ( 4.0) =,6.1905,Pearson Chi-square ( 4.0) =,4.4286,Pearson Chi-square ( 4.0) =,4.4286
1,p-value =,0.1854,p-value =,0.3511,p-value =,0.3511
2,Cramer's V =,0.5563,Cramer's V =,0.4706,Cramer's V =,0.4706


-----------------------------------------------------------------------
Number  92


,Chi-square test,results,Chi-square test,results,Chi-square test,results
0,Pearson Chi-square ( 2.0) =,10.0000,Pearson Chi-square ( 2.0) =,7.9167,Pearson Chi-square ( 2.0) =,7.2222
1,p-value =,0.0067,p-value =,0.0191,p-value =,0.0270
2,Cramer's V =,1.0000,Cramer's V =,0.8898,Cramer's V =,0.8498


-----------------------------------------------------------------------


# Chi-square test of independence of variables in a contingency table. (for references)

In [168]:
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

def chi2_test(obs, exp):
    l1 =exp.to_numpy()
    l2= obs.to_numpy()
    for i in range(len(obs)):
        data = [l1[i], l2[i]]
        display(data)
        stat, p, dof, expected = chi2_contingency(data)

        # interpret test-statistic
        prob = 0.95
        critical = chi2.ppf(prob, dof)
        print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
        if abs(stat) >= critical:
            print('Dependent (reject H0)')
        else:
            print('Independent (fail to reject H0)')

       # interpret p-value
        alpha = 1.0 - prob
        print('significance=%.3f, p=%.3f' % (alpha, p))
        if p <= alpha:
            print('Dependent (reject H0)')
        else:
            print('Independent (fail to reject H0)')
        print('')
   

    return 

# Euler Angle

In [45]:
euler=['arEulerAngleFaceX', 'arEulerAngleFaceY', 'arEulerAngleFaceZ']

In [46]:
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
brow =list(df[df['eventName'].str.contains('presentedLiveItem')].index)  
erow= list(df[df['eventName'].str.contains('completedLiveItem')].index)

EA = []
for i in range(len(erow)):
    sub = df.iloc[brow[i]+1:erow[i]][euler].dropna()
    summary = sub.describe()[1:3]
    summary.loc['Kurtosis'] = kurtosis(sub)
    summary.loc['Skew'] = skew(sub)
    display(summary)
    EA.append(summary)

,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.016042,0.105156,0.056655
std,0.073443,0.061616,0.035063
Kurtosis,-1.328017,-0.288318,-0.679476
Skew,-0.043950,-0.734964,-0.009703


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.040761,0.205920,-0.002911
std,0.053658,0.162100,0.035259
Kurtosis,1.712758,0.551409,-0.670923
Skew,-1.171197,1.358672,-0.496988


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.010905,0.085573,0.020747
std,0.059262,0.111691,0.088701
Kurtosis,0.719910,-1.065274,-0.626506
Skew,0.910714,-0.614036,0.482929


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.031980,0.027270,0.144209
std,0.093615,0.077202,0.104649
Kurtosis,1.515859,-0.888548,1.198509
Skew,0.789121,-0.858333,1.473389


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.098668,0.076826,-0.045378
std,0.045777,0.063790,0.074273
Kurtosis,-0.191110,-0.313478,0.131153
Skew,-0.694129,-1.010555,-1.317651


...

,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.153623,0.113402,0.200321
std,0.082426,0.109593,0.194279
Kurtosis,-0.440196,-1.073630,0.137603
Skew,-0.047247,0.150437,0.912630


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.003756,0.096548,0.024924
std,0.036564,0.034382,0.021185
Kurtosis,-0.234550,-0.807045,-0.700486
Skew,0.444920,0.364545,0.364065


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.053671,0.067290,-0.028022
std,0.027051,0.093791,0.105272
Kurtosis,1.411465,-0.710593,0.141894
Skew,0.881281,-0.946447,-1.317226


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.049345,0.092111,-0.025921
std,0.024402,0.023582,0.017721
Kurtosis,2.305433,-0.197257,-0.718545
Skew,1.496058,0.695441,-0.487519


,arEulerAngleFaceX,arEulerAngleFaceY,arEulerAngleFaceZ
mean,0.129143,0.106811,-0.020187
std,0.097436,0.120918,0.047884
Kurtosis,3.828935,8.584933,13.232644
Skew,2.084855,2.624298,-3.155102


In [ ]:
EulerAngle = pd.concat(EA)
#EulerAngle.to_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Analysis/EulerAngleStats.csv', index=False)